In [209]:
import pandas as pd
import matplotlib.pyplot  as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Importando Excel de defeitos e transformando para Dataframe Pandas

In [152]:
dados = pd.read_excel('../dados/SetoutDefect_20250103_142537_734275983 (1).xlsx', nrows=38625)
df = pd.DataFrame(dados)

In [153]:
df.columns

Index(['Fábrica', 'Fabrica – Linha', 'Linha', 'Máquina de produção',
       'Ordem de trabalhos', 'Ref. do artigo', 'Artigo',
       'Ordem de sub-trabalho', 'Produto final', 'Lote', 'Pallet n.º',
       'Definir data', 'Localização', 'Localização do setout', 'Bloquear', 'S',
       'Gota', 'Gota0', 'Macho', 'n.º de defeitos', 'Defeito repetitivo',
       'Amostra n.º', '% de defeitos', 'Ref. do molde', 'Número de série',
       'Molde', 'Camada', 'Número de defeitos por camada', 'Código', 'Defeito',
       'Cor', 'Criticidade', 'Criticidade1', 'Equipe', 'Operador', 'Nqa',
       'Estado', 'A verificar', 'Comentário', 'Data de criação',
       'Chave primária', 'Definir dia', 'Dia de criação', 'Turno',
       'Definir chave primária'],
      dtype='object')

In [154]:
df

,Fábrica,Fabrica – Linha,Linha,Máquina de produção,Ordem de trabalhos,Ref. do artigo,Artigo,Ordem de sub-trabalho,Produto final,Lote,...,Nqa,Estado,A verificar,Comentário,Data de criação,Chave primária,Definir dia,Dia de criação,Turno,Definir chave primária
0,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:03:07,2517600,2024-01-01,2024-01-01,1,872218
1,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:03:07,2517601,2024-01-01,2024-01-01,1,872218
2,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:57:09,2517738,2024-01-01,2024-01-01,1,872263
3,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:57:09,2517739,2024-01-01,2024-01-01,1,872263
4,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:57:09,2517740,2024-01-01,2024-01-01,1,872263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38620,Wheaton,Wheaton - B1,B1,Máquina IS,194121,C0516.0000R,MB -1377-N1N | F | NH | B | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-06-30 23:20:03,3160120,2024-06-30,2024-06-30,3,1091441
38621,Wheaton,Wheaton - B1,B1,Máquina IS,194121,C0516.0000R,MB -1377-N1N | F | NH | B | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-06-30 23:20:03,3160126,2024-06-30,2024-06-30,3,1091441
38622,Wheaton,Wheaton - B1,B1,Máquina IS,194121,C0516.0000R,MB -1377-N1N | F | NH | B | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-06-30 23:20:03,3160122,2024-06-30,2024-06-30,3,1091441
38623,Wheaton,Wheaton - B1,B1,Máquina IS,194121,C0516.0000R,MB -1377-N1N | F | NH | B | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-06-30 23:20:03,3160127,2024-06-30,2024-06-30,3,1091441


In [155]:
artigos_nuniques_ordem = df[df.groupby('Ordem de trabalhos')['Ref. do artigo'].transform('nunique') > 1]
artigos_nuniques_ordem

,Fábrica,Fabrica – Linha,Linha,Máquina de produção,Ordem de trabalhos,Ref. do artigo,Artigo,Ordem de sub-trabalho,Produto final,Lote,...,Nqa,Estado,A verificar,Comentário,Data de criação,Chave primária,Definir dia,Dia de criação,Turno,Definir chave primária


In [156]:
artigos_ordem = df.groupby('Ordem de trabalhos')['Ref. do artigo'].unique()
artigos_ordem

Ordem de trabalhos
192970    [C0516.0000R]
193112    [C1164.0000R]
193164    [C1280.0000R]
193206    [C1284.0000R]
193207    [C1298.0000R]
193208    [C0104.0000R]
193209    [C0509.0000R]
193342    [C0516.0000R]
193368    [C1137.0000R]
193459    [C0068.0000R]
193462    [C1284.0000R]
193463    [C1298.0000R]
193533    [C1165.0000R]
193535    [C1332.0000V]
193537    [C1203.0000R]
193594    [C1137.0000R]
193619    [C0121.0000R]
193671    [C0593.0000R]
193686    [C1239.0000V]
193688    [C1349.0000V]
193689    [C1299.0000R]
193692    [C0139.0000R]
193694    [C0529.0000R]
193782    [C1265.0000R]
193783    [C1215.0000R]
193784    [C1203.0000R]
193852    [C1267.0000R]
193882    [C1268.0000R]
193937    [C1172.0000R]
193972    [C1203.0000R]
193973    [C1165.0000R]
194029    [C0132.0000R]
194057    [C1164.0000R]
194113    [C0104.0000R]
194121    [C0516.0000R]
Name: Ref. do artigo, dtype: object

# Criando novo Dataframe para trabalhar com os defeitos por data

### Criando coluna ``Data``, somente com data, sem o horário do registro

In [157]:
df_por_data = df.copy()

df_por_data['Data'] = df_por_data['Data de criação'].dt.date

df_por_data.groupby(['Ordem de trabalhos', 'Ref. do artigo','Data'])[['Defeito']].count()

Defeito
Ordem de trabalhos Ref. do artigo Data               
192970             C0516.0000R    2024-01-03      162
                                  2024-01-04      249
                                  2024-01-05      268
                                  2024-01-06      242
                                  2024-01-07      232
...                                               ...
194121             C0516.0000R    2024-06-27      218
                                  2024-06-28      218
                                  2024-06-29      213
                                  2024-06-30      256
                                  2024-07-01        1

[211 rows x 1 columns]

### Criando Dataframe multiíndice, agrupado por `Ordem de trabalhos`, `Data` e `Defeito`, contando a frequência de cada defeito 

In [158]:
frequencia_defeitos = df_por_data.groupby(['Ordem de trabalhos', 'Ref. do artigo', 'Data', 'Defeito']).size().reset_index(name='Quantidade')
frequencia_defeitos = frequencia_defeitos.set_index(['Ordem de trabalhos', 'Ref. do artigo', 'Data', 'Defeito'])
frequencia_defeitos = frequencia_defeitos.sort_index()

frequencia_defeitos

Quantidade
Ordem de trabalhos Ref. do artigo Data       Defeito                              
192970             C0516.0000R    2024-01-03 Cava no ombro                      10
                                             Corpo torto                         6
                                             Costura                             3
                                             Costura desencontrada               7
                                             Dobra                              10
...                                                                            ...
194121             C0516.0000R    2024-06-30 Sujo de óleo                       58
                                             Terminação deformada                1
                                             Terminação suja                     5
                                             Trinca por choque térmico          17
                                  2024-07-01 Bolha                               1

[4687 rows x 1 columns]

# Criando um Dataframe resumo de defeitos

## Calculando o total de defeitos por cada ``Ordem`` e qual o defeito mais frequente de cada ``Ordem``

### Criando colunas `Data_Inicio` e `Data_Fim` para monitorar produtividades 

In [159]:
frequencia_defeitos_reset = frequencia_defeitos.reset_index()

inicio_fim_ordem = frequencia_defeitos_reset.groupby('Ordem de trabalhos').agg(
    Data_Inicio=('Data', 'min'),
    Data_Fim=('Data', 'max')
)

### Calculando a Quantidade e Frequência

In [160]:
frequencia_defeitos_reset = frequencia_defeitos.reset_index()

ref_artigo_por_ordem = frequencia_defeitos_reset.groupby('Ordem de trabalhos')['Ref. do artigo'].agg(lambda x: ', '.join(x.unique()))

total_defeitos_por_ordem = frequencia_defeitos.groupby('Ordem de trabalhos')['Quantidade'].sum()

defeito_mais_frequente_por_ordem = frequencia_defeitos.groupby('Ordem de trabalhos')['Quantidade'].idxmax().map(lambda x: x[3])

resumo_defeitos_por_ordem = pd.DataFrame({
    'Ref. do Artigo': ref_artigo_por_ordem,
    'Total de Defeitos': total_defeitos_por_ordem,
    'Defeito Mais Frequente': defeito_mais_frequente_por_ordem
})

resumo_defeitos_por_ordem = resumo_defeitos_por_ordem.join(inicio_fim_ordem)

resumo_defeitos_por_ordem.sort_values('Total de Defeitos', ascending=False, inplace=True)

resumo_defeitos_por_ordem_reset = resumo_defeitos_por_ordem.reset_index()
resumo_defeitos_por_ordem_reset.set_index(['Ordem de trabalhos', 'Ref. do Artigo'])

,,Total de Defeitos,Defeito Mais Frequente,Data_Inicio,Data_Fim
Ordem de trabalhos,Ref. do Artigo,,,,
193342,C0516.0000R,3568,Sujo de óleo,2024-02-21,2024-03-11
193164,C1280.0000R,2574,Sujo de óleo,2024-01-11,2024-01-23
193782,C1265.0000R,2263,Dobra,2024-05-15,2024-05-25
193207,C1298.0000R,1953,Sujo de óleo,2024-01-28,2024-02-06
192970,C0516.0000R,1941,Sujo de óleo,2024-01-03,2024-01-11
193784,C1203.0000R,1930,Sujo de óleo,2024-05-04,2024-05-15
193209,C0509.0000R,1663,Sujo de óleo,2024-02-08,2024-02-14
193368,C1137.0000R,1581,Sujo de óleo,2024-02-14,2024-02-21
193882,C1268.0000R,1521,Sujo de óleo,2024-06-08,2024-06-15


### Criando coluna `Dias de Ordem` para visualizar a quantidade de dias da ordem 

In [161]:
resumo_defeitos_por_ordem['Data_Inicio'] = pd.to_datetime(resumo_defeitos_por_ordem['Data_Inicio'], errors='coerce')
resumo_defeitos_por_ordem['Data_Fim'] = pd.to_datetime(resumo_defeitos_por_ordem['Data_Fim'], errors='coerce')

resumo_defeitos_por_ordem['Dias de Ordem'] = (resumo_defeitos_por_ordem['Data_Fim'] - resumo_defeitos_por_ordem['Data_Inicio']).dt.days

resumo_defeitos_por_ordem

,Ref. do Artigo,Total de Defeitos,Defeito Mais Frequente,Data_Inicio,Data_Fim,Dias de Ordem
Ordem de trabalhos,,,,,,
193342,C0516.0000R,3568,Sujo de óleo,2024-02-21,2024-03-11,19
193164,C1280.0000R,2574,Sujo de óleo,2024-01-11,2024-01-23,12
193782,C1265.0000R,2263,Dobra,2024-05-15,2024-05-25,10
193207,C1298.0000R,1953,Sujo de óleo,2024-01-28,2024-02-06,9
192970,C0516.0000R,1941,Sujo de óleo,2024-01-03,2024-01-11,8
193784,C1203.0000R,1930,Sujo de óleo,2024-05-04,2024-05-15,11
193209,C0509.0000R,1663,Sujo de óleo,2024-02-08,2024-02-14,6
193368,C1137.0000R,1581,Sujo de óleo,2024-02-14,2024-02-21,7
193882,C1268.0000R,1521,Sujo de óleo,2024-06-08,2024-06-15,7


### Gráfico relacionando tempo de ordem e total de defeitos

In [162]:
fig_scatter = px.scatter(resumo_defeitos_por_ordem,
                         x='Dias de Ordem',
                         y='Total de Defeitos',
                         title='Dias de Ordem vs Total de Defeitos',
                         labels={'Dias de Ordem': 'Dias de Ordem', 'Total de Defeitos': 'Total de Defeitos'},
                         color='Total de Defeitos',
                         color_continuous_scale='redor',
                         size='Dias de Ordem'
                         )
fig_scatter.show()

## Calculando o total de defeitos por cada ``Data`` e qual o defeito mais frequente de cada ``Data``

In [163]:
frequencia_defeitos_reset = frequencia_defeitos.reset_index()

total_defeitos_por_data = frequencia_defeitos.groupby('Data')['Quantidade'].sum()

defeito_mais_frequente_por_data_idx = frequencia_defeitos.groupby('Data')['Quantidade'].idxmax()

defeito_mais_frequente_por_data = defeito_mais_frequente_por_data_idx.apply(lambda idx: idx[3])

resumo_defeitos_por_data = pd.DataFrame({
    'Total de Defeitos': total_defeitos_por_data,
    'Defeito Mais Frequente': defeito_mais_frequente_por_data,
})

resumo_defeitos_por_data = resumo_defeitos_por_data.sort_index()

resumo_defeitos_por_data

,Total de Defeitos,Defeito Mais Frequente
Data,,
2024-01-01,159,Sujo de óleo
2024-01-02,203,Sujo de óleo
2024-01-03,222,Sujo de óleo
2024-01-04,249,Sujo de óleo
2024-01-05,268,Sujo de óleo
...,...,...
2024-06-27,218,Sujo de óleo
2024-06-28,218,Sujo de óleo
2024-06-29,213,Sujo de óleo


# Trabalhando com Dataframe de `medidas da gota` e Dataframe com `total de defeitos` e `defeito mais frequente` por `Ordem` e `Data`

## Importando Dataframe com as medidas da gota de cada ordem

In [164]:
medidas_ordem = pd.read_excel('../dados/media_mensal_ordem_completa.xlsx')

medidas_ordem.set_index(['Ordem de prod', 'Data de prod'], inplace=True)

## Unindo os dois Dataframes

### Resetando os índices dos dataframes para união, renomeando coluna `Data` do conjunto `resumo_defeitos_por_data_reset` para o mesmo do conjunto `medidas_ordem`

In [165]:
resumo_defeitos_por_data_reset = resumo_defeitos_por_data.reset_index()
resumo_defeitos_por_data_reset.rename(columns={'Data': 'Data de prod'}, inplace=True)

medidas_ordem_reset = medidas_ordem.reset_index()

### Retransformando colunas `Data de prod` para datetime após resetar o index, setando o index do conjunto `resumo_defeitos_por_ordem_reset` para união 

In [166]:
medidas_ordem_reset['Data de prod'] = pd.to_datetime(medidas_ordem_reset['Data de prod'])
resumo_defeitos_por_data_reset['Data de prod'] = pd.to_datetime(resumo_defeitos_por_data_reset['Data de prod'])

resumo_defeitos_por_ordem_reset = resumo_defeitos_por_ordem_reset.set_index('Ordem de trabalhos')

df_resumo_geral = pd.merge(
    medidas_ordem_reset, 
    resumo_defeitos_por_data_reset, 
    on='Data de prod', 
    how='inner'
)

### Adicionando coluna `Ref. do Artigo` e a transformando em índice

In [167]:
df_resumo_geral = pd.merge(
    df_resumo_geral, 
    resumo_defeitos_por_ordem_reset[['Ref. do Artigo']], 
    left_on='Ordem de prod', 
    right_index=True, 
    how='left'
)

df_resumo_geral = df_resumo_geral.set_index(['Ordem de prod', 'Ref. do Artigo', 'Data de prod'])

df_resumo_geral.sort_index(inplace=True)

### Checando valores nulos e tipos de dados

In [168]:
df_resumo_geral.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 200 entries, (192970, 'C0516.0000R', Timestamp('2024-01-03 00:00:00')) to (194121, 'C0516.0000R', Timestamp('2024-06-30 00:00:00'))
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gotas cortadas (toneladas)  200 non-null    float64
 1   Peso médio (g)              200 non-null    float64
 2   Cortes por minuto           200 non-null    float64
 3   Comprimento médio (mm)      200 non-null    float64
 4   Diâmetro                    200 non-null    float64
 5   Temperatura (F°)            200 non-null    float64
 6   Tras-PV (F°)                200 non-null    float64
 7   Fritas-PV (F°)              200 non-null    float64
 8   Agit-PV (F°)                200 non-null    float64
 9   Meio-PV (F°)                200 non-null    float64
 10  Frente-PV (F°)              200 non-null    float64
 11  Total de Defeitos           200 non-nul

### Renomendo colunas para facilitar a visualização

In [169]:
df_resumo_geral = df_resumo_geral.rename(columns={'Total de Defeitos': 'Total de Defeitos no Dia', 'Defeito Mais Frequente': 'Defeito Mais Frequente do Dia'})

### Dataframe de Total de Defeitos e Defeito Mais Frequente

In [170]:
df_resumo_geral

Gotas cortadas (toneladas)  \
Ordem de prod Ref. do Artigo Data de prod                               
192970        C0516.0000R    2024-01-03                        20.948   
                             2024-01-04                        21.022   
                             2024-01-05                        21.449   
                             2024-01-06                        21.526   
                             2024-01-07                        21.439   
...                                                               ...   
194121        C0516.0000R    2024-06-26                        21.412   
                             2024-06-27                        21.447   
                             2024-06-28                        21.491   
                             2024-06-29                        21.419   
                             2024-06-30                        21.548   

                                           Peso médio (g)  Cortes por minuto  \
Ordem de prod Ref. do Artigo Data de prod                                      
192970        C0516.0000R    2024-01-03             169.3               90.0   
                             2024-01-04             169.7               89.0   
                             2024-01-05             173.6               86.0   
                             2024-01-06             170.2               86.0   
                             2024-01-07             171.7               86.7   
...                                                   ...                ...   
194121        C0516.0000R    2024-06-26             169.0               88.0   
                             2024-06-27             168.9               88.0   
                             2024-06-28             169.3               88.0   
                             2024-06-29             168.8               88.0   
                             2024-06-30             169.2               88.9   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Temperatura (F°)  Tras-PV (F°)  \
Ordem de prod Ref. do Artigo Data de prod                                   
192970        C0516.0000R    2024-01-03             2505.02        2341.1   
                             2024-01-04             3094.52        2349.6   
                             2024-01-05             3369.74        2350.0   
                             2024-01-06             3335.18        2350.0   
                             2024-01-07             3340.94        2349.6   
...                                                     ...           ...   
194121        C0516.0000R    2024-06-26             1940.00        2340.0   
                             2024-06-27             2056.28        2340.0   
                             2024-06-28             2015.42        2340.0   
                             2024-06-29             2020.28        2340.0   
                             2024-06-30             2055.56        2340.0   

                      

# "Prevendo" a temperatura da gota

### Transformando todos os dados das colunas de temperatura para float64

In [198]:
cols = ['Tras-PV (F°)', 'Fritas-PV (F°)', 'Agit-PV (F°)', 'Meio-PV (F°)', 'Frente-PV (F°)']
df_resumo_geral[cols] = df_resumo_geral[cols].apply(pd.to_numeric, errors='coerce')
print(df_resumo_geral[cols].dtypes)

Tras-PV (F°)      float64
Fritas-PV (F°)    float64
Agit-PV (F°)      float64
Meio-PV (F°)      float64
Frente-PV (F°)    float64
dtype: object


### Gráfico de regressão das temperaturas

In [211]:
x = np.arange(len(cols)) 
y = df_resumo_geral[cols].iloc[0].values 

slope, intercept = np.polyfit(x, y, 1)
linha_regressao = slope * x + intercept

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y, mode='markers+lines', name='Temperaturas (dados reais)', 
                         line=dict(color='blue'), marker=dict(color='blue', size=10)))

fig.add_trace(go.Scatter(x=x, y=linha_regressao, mode='lines', name=f'Linha de Regressão\n y = {slope:.2f}x + {intercept:.2f}', 
                         line=dict(color='red', dash='dash')))

fig.update_layout(
    title='Taxa de Variação nas Temperaturas',
    xaxis=dict(tickmode='array', tickvals=x, ticktext=cols, title='Posição (Colunas)'),
    yaxis=dict(title='Temperatura (°F)'),
    showlegend=True,
    template='plotly',
    xaxis_tickangle=-45,
    margin=dict(l=40, r=40, t=40, b=100)
)

fig.show()


> A linha de regressão mostra a tendência geral das temperaturas 'PV' nas colunas do DataFrame. A taxa de variação (inclinação) é **-43.07°F por posição**, o que significa que, em média, as temperaturas diminuem 43,07°F a cada etapa do feeder.


### Calculando próxima temperatura (da gota) com base na linha de regressão

In [200]:
def prever_proxima_temp(row):
    try:
        x = np.arange(len(cols))
        y = row[cols].astype(float).values
        slope, intercept = np.polyfit(x, y, 1)
        proxima_temp = slope * (len(cols)) + intercept
        return proxima_temp
    except Exception as e:
        print(f"Erro com os dados: x={x}, y={y}, erro={e}")
        return np.nan

df_resumo_geral['Proxima_Temperatura'] = df_resumo_geral.apply(prever_proxima_temp, axis=1)

In [201]:
df_resumo_geral

Gotas cortadas (toneladas)  \
Ordem de prod Ref. do Artigo Data de prod                               
192970        C0516.0000R    2024-01-03                        20.948   
                             2024-01-04                        21.022   
                             2024-01-05                        21.449   
                             2024-01-06                        21.526   
                             2024-01-07                        21.439   
...                                                               ...   
194121        C0516.0000R    2024-06-26                        21.412   
                             2024-06-27                        21.447   
                             2024-06-28                        21.491   
                             2024-06-29                        21.419   
                             2024-06-30                        21.548   

                                           Peso médio (g)  Cortes por minuto  \
Ordem de prod Ref. do Artigo Data de prod                                      
192970        C0516.0000R    2024-01-03             169.3               90.0   
                             2024-01-04             169.7               89.0   
                             2024-01-05             173.6               86.0   
                             2024-01-06             170.2               86.0   
                             2024-01-07             171.7               86.7   
...                                                   ...                ...   
194121        C0516.0000R    2024-06-26             169.0               88.0   
                             2024-06-27             168.9               88.0   
                             2024-06-28             169.3               88.0   
                             2024-06-29             168.8               88.0   
                             2024-06-30             169.2               88.9   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Temperatura (F°)  Tras-PV (F°)  \
Ordem de prod Ref. do Artigo Data de prod                                   
192970        C0516.0000R    2024-01-03             2126.97        2341.1   
                             2024-01-04             2118.92        2349.6   
                             2024-01-05             2125.00        2350.0   
                             2024-01-06             2125.40        2350.0   
                             2024-01-07             2126.76        2349.6   
...                                                     ...           ...   
194121        C0516.0000R    2024-06-26             2154.44        2340.0   
                             2024-06-27             2152.32        2340.0   
                             2024-06-28             2145.60        2340.0   
                             2024-06-29             2145.60        2340.0   
                             2024-06-30             2148.00        2340.0   

                      

### Substituição das colunas

In [202]:
df_resumo_geral.drop(columns='Temperatura (F°)', inplace=True)

In [203]:
df_resumo_geral.rename(columns={'Proxima_Temperatura': 'Temperatura (F°)'}, inplace=True)

### Reordenando Dataframe

In [204]:
nova_ordem = [
    'Gotas cortadas (toneladas)', 
    'Peso médio (g)', 
    'Cortes por minuto', 
    'Comprimento médio (mm)', 
    'Diâmetro', 
    'Temperatura (F°)', 
    'Tras-PV (F°)', 
    'Fritas-PV (F°)', 
    'Agit-PV (F°)', 
    'Meio-PV (F°)', 
    'Frente-PV (F°)', 
    'Total de Defeitos no Dia', 
    'Defeito Mais Frequente do Dia'
]

df_resumo_geral = df_resumo_geral[nova_ordem]

In [205]:
df_resumo_geral

Gotas cortadas (toneladas)  \
Ordem de prod Ref. do Artigo Data de prod                               
192970        C0516.0000R    2024-01-03                        20.948   
                             2024-01-04                        21.022   
                             2024-01-05                        21.449   
                             2024-01-06                        21.526   
                             2024-01-07                        21.439   
...                                                               ...   
194121        C0516.0000R    2024-06-26                        21.412   
                             2024-06-27                        21.447   
                             2024-06-28                        21.491   
                             2024-06-29                        21.419   
                             2024-06-30                        21.548   

                                           Peso médio (g)  Cortes por minuto  \
Ordem de prod Ref. do Artigo Data de prod                                      
192970        C0516.0000R    2024-01-03             169.3               90.0   
                             2024-01-04             169.7               89.0   
                             2024-01-05             173.6               86.0   
                             2024-01-06             170.2               86.0   
                             2024-01-07             171.7               86.7   
...                                                   ...                ...   
194121        C0516.0000R    2024-06-26             169.0               88.0   
                             2024-06-27             168.9               88.0   
                             2024-06-28             169.3               88.0   
                             2024-06-29             168.8               88.0   
                             2024-06-30             169.2               88.9   

                                           Comprimento médio (mm)  Diâmetro  \
Ordem de prod Ref. do Artigo Data de prod                                     
192970        C0516.0000R    2024-01-03                     110.4      32.5   
                             2024-01-04                      85.6      18.1   
                             2024-01-05                     140.5      32.9   
                             2024-01-06                     144.7      32.7   
                             2024-01-07                     141.5      30.2   
...                                                           ...       ...   
194121        C0516.0000R    2024-06-26                     124.3      32.6   
                             2024-06-27                     120.2      33.6   
                             2024-06-28                     124.1      34.2   
                             2024-06-29                     120.0      33.5   
                             2024-06-30                     122.9      34.6   

                                           Temperatura (F°)  Tras-PV (F°)  \
Ordem de prod Ref. do Artigo Data de prod                                   
192970        C0516.0000R    2024-01-03             2126.97        2341.1   
                             2024-01-04             2118.92        2349.6   
                             2024-01-05             2125.00        2350.0   
                             2024-01-06             2125.40        2350.0   
                             2024-01-07             2126.76        2349.6   
...                                                     ...           ...   
194121        C0516.0000R    2024-06-26             2154.44        2340.0   
                             2024-06-27             2152.32        2340.0   
                             2024-06-28             2145.60        2340.0   
                             2024-06-29             2145.60        2340.0   
                             2024-06-30             2148.00        2340.0   

                      

### Analisando se a coluna `Total de Defeitos no Dia` é afetada pela `Temperatura (F°)`

In [206]:
fig = px.scatter(
    df_resumo_geral,
    x="Temperatura (F°)", 
    y="Total de Defeitos no Dia",
    title="Dispersão: Total de Defeitos vs Temperatura da Gota",
    labels={"Temperatura (F°)": "Temperatura (F°)", "Total de Defeitos no Dia": "Total de Defeitos"},
    hover_data=["Defeito Mais Frequente do Dia"],
    color='Temperatura (F°)',
    color_continuous_scale='redor',
    size='Total de Defeitos no Dia'
)

fig.show()

### Resetando índice para criação do gráfico

In [207]:
df_resumo_geral_reset = df_resumo_geral.reset_index()

### Gráfico de quantidade de defeitos por cada variável de medida da gota, visualizar como cada variável afeta a quantidade de defeitos

In [212]:
df_resumo_geral_reset = df_resumo_geral_reset.sort_values('Data de prod', ascending=False)

variaveis = [
    {"coluna": "Temperatura (F°)", "cor": "red", "nome": "Temperatura (F°)"},
    {"coluna": "Diâmetro", "cor": "green", "nome": "Diâmetro"},
    {"coluna": "Comprimento médio (mm)", "cor": "orange", "nome": "Comprimento Médio (mm)"},
    {"coluna": "Peso médio (g)", "cor": "purple", "nome": "Peso Médio (g)"}
]

for var in variaveis:
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_resumo_geral_reset["Data de prod"],
            y=df_resumo_geral_reset["Total de Defeitos no Dia"],
            mode="lines+markers",
            name="Defeitos no Dia",
            line=dict(color="blue")
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df_resumo_geral_reset["Data de prod"],
            y=df_resumo_geral_reset[var["coluna"]],
            mode="lines+markers",
            name=var["nome"],
            line=dict(color=var["cor"], dash="dash"),
            yaxis="y2"
        )
    )

    fig.update_layout(
        title=f"Total de Defeitos por Data e Relação com {var['nome']}",
        xaxis=dict(title="Data"),
        yaxis=dict(
            title="Quantidade de Defeitos",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue")
        ),
        yaxis2=dict(
            title=var["nome"],
            titlefont=dict(color=var["cor"]),
            tickfont=dict(color=var["cor"]),
            overlaying="y",
            side="right"
        ),
        legend_title="Legenda",
        template="plotly_dark"
    )

    fig.show()

# Criando Dataframe com todas as informações de defeitos e medidas das gotas, para tratar cada defeito particularmente

In [182]:
frequencia_defeitos_reset.sort_values('Quantidade', ascending=False)

,Ordem de trabalhos,Ref. do artigo,Data,Defeito,Quantidade
3251,193782,C1265.0000R,2024-05-16,Dobra,147
962,193207,C1298.0000R,2024-02-04,Sujo de óleo,139
294,193112,C1164.0000R,2024-01-02,Sujo de óleo,125
2185,193463,C1298.0000R,2024-03-17,Sujo de óleo,119
1233,193209,C0509.0000R,2024-02-13,Sujo de óleo,115
...,...,...,...,...,...
1730,193368,C1137.0000R,2024-02-15,Frio,1
1734,193368,C1137.0000R,2024-02-15,Largura +,1
3554,193784,C1203.0000R,2024-05-05,Trinca por choque térmico,1
1746,193368,C1137.0000R,2024-02-15,Racho no fundo,1


### Unindo Dataframes

In [183]:
frequencia_defeitos_reset.rename(columns={
    'Ordem de trabalhos': 'Ordem de prod',
    'Ref. do artigo': 'Ref. do Artigo',
    'Data': 'Data de prod'
}, inplace=True)

frequencia_defeitos_reset['Data de prod'] = pd.to_datetime(frequencia_defeitos_reset['Data de prod'])
df_resumo_geral_reset['Data de prod'] = pd.to_datetime(df_resumo_geral_reset['Data de prod'])

df_combinado_defeitos_medidas = pd.merge(frequencia_defeitos_reset, df_resumo_geral_reset, 
                     on=['Ordem de prod', 'Ref. do Artigo', 'Data de prod'], 
                     how='left')

### Tratando linha com valores NaN no Dataframe

In [184]:
df_combinado_defeitos_medidas = df_combinado_defeitos_medidas.dropna(subset=['Total de Defeitos no Dia'])
df_combinado_defeitos_medidas['Total de Defeitos no Dia'] = df_combinado_defeitos_medidas['Total de Defeitos no Dia'].astype(int)

### Setando os índices 

In [185]:
colunas_indice = ['Ordem de prod', 'Ref. do Artigo', 'Data de prod', 'Total de Defeitos no Dia', 'Defeito Mais Frequente do Dia', 'Temperatura (F°)', 'Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro', 'Defeito']

df_combinado_defeitos_medidas = df_combinado_defeitos_medidas.drop(columns=['Gotas cortadas (toneladas)', 'Cortes por minuto', 'Tras-PV (F°)', 'Fritas-PV (F°)', 'Agit-PV (F°)', 'Meio-PV (F°)', 'Frente-PV (F°)'])

df_combinado_defeitos_medidas = df_combinado_defeitos_medidas.set_index(colunas_indice)

In [186]:
df_combinado_defeitos_medidas

Quantidade
Ordem de prod Ref. do Artigo Data de prod Total de Defeitos no Dia Defeito Mais Frequente do Dia Temperatura (F°) Peso médio (g) Comprimento médio (mm) Diâmetro Defeito                              
192970        C0516.0000R    2024-01-03   222                      Sujo de óleo                  2126.97          169.3          110.4                  32.5     Cava no ombro                      10
                                                                                                                                                                 Corpo torto                         6
                                                                                                                                                                 Costura                             3
                                                                                                                                                                 Costura desencontrada               7
                                                                                                                                                                 Dobra                              10
...                                                                                                                                                                                                ...
194121        C0516.0000R    2024-06-30   256                      Sujo de óleo                  2148.00          169.2          122.9                  34.6     Rugas                              25
                                                                                                                                                                 Sujo de óleo                       58
                                                                                                                                                                 Terminação deformada                1
                                                                                                                                                                 Terminação suja                     5
                                                                                                                                                                 Trinca por choque térmico          17

[4519 rows x 1 columns]

## Criando Dataframe para análise completa dos defeitos e medidas 

### `pivot_table` cria uma tabela em que cada defeito será uma coluna, e as variáveis numéricas são agrupadas nas linhas. 

A função aggfunc='sum' soma as quantidades de defeitos para cada combinação de parâmetros.

In [187]:
df_defeitos_medidas_geral = df_combinado_defeitos_medidas.pivot_table(
    values='Quantidade', 
    index=['Ordem de prod', 'Ref. do Artigo', 'Data de prod', 'Temperatura (F°)', 'Peso médio (g)', 
           'Comprimento médio (mm)', 'Diâmetro', 'Total de Defeitos no Dia', 'Defeito Mais Frequente do Dia'],
    columns='Defeito', 
    aggfunc='sum',
    fill_value=0
)

### Apagando a linha/coluna 'Defeito'

In [188]:
df_defeitos_medidas_geral.columns.name = None

### Ordenando por colunas com mais defeitos

In [189]:
colunas_ordered = df_defeitos_medidas_geral.sum(axis=0).sort_values(ascending=False).index
df_defeitos_medidas_geral = df_defeitos_medidas_geral[colunas_ordered]

In [190]:
df_defeitos_medidas_geral

Sujo de óleo  \
Ordem de prod Ref. do Artigo Data de prod Temperatura (F°) Peso médio (g) Comprimento médio (mm) Diâmetro Total de Defeitos no Dia Defeito Mais Frequente do Dia                 
192970        C0516.0000R    2024-01-03   2126.97          169.3          110.4                  32.5     222                      Sujo de óleo                             31   
                             2024-01-04   2118.92          169.7          85.6                   18.1     249                      Sujo de óleo                             49   
                             2024-01-05   2125.00          173.6          140.5                  32.9     268                      Sujo de óleo                             49   
                             2024-01-06   2125.40          170.2          144.7                  32.7     242                      Sujo de óleo                             73   
                             2024-01-07   2126.76          171.7          141.5                  30.2     232                      Sujo de óleo                             49   
...                                                                                                                                                                        ...   
194121        C0516.0000R    2024-06-26   2154.44          169.0          124.3                  32.6     258                      Sujo de óleo                             40   
                             2024-06-27   2152.32          168.9          120.2                  33.6     218                      Sujo de óleo                             75   
                             2024-06-28   2145.60          169.3          124.1                  34.2     218                      Sujo de óleo                             54   
                             2024-06-29   2145.60          168.8          120.0                  33.5     213                      Sujo de óleo                             54   
                             2024-06-30   2148.00          169.2          122.9                  34.6     256                      Sujo de óleo                             58   

                                                                                                                                                                  Rugas  \
Ordem de prod Ref. do Artigo Data de prod Temperatura (F°) Peso médio (g) Comprimento médio (mm) Diâmetro Total de Defeitos no Dia Defeito Mais Frequente do Dia          
192970        C0516.0000R    2024-01-03   2126.97          169.3          110.4                  32.5     222                      Sujo de óleo                      23   
                             2024-01-04   2118.92          169.7          85.6                   18.1     249                      Sujo de óleo                      15   
                             2024-01-05   2125.00          173.6          140.5                  32.9     268                      Sujo de óleo                      25   
                             2024-01-06   2125.40          170.2          144.7                  32.7     242                      Sujo de óleo                      20   
                             2024-01-07   2126.76          171.7          141.5                  30.2     232                      Sujo de óleo                      24   
...                                                                                                                                                                 ...   
194121        C0516.0000R    2024-06-26   2154.44          169.0          124.3                  32.6     258                      Sujo de óleo                      39   
                             2024-06-27   2152.32          168.9          120.2                  33.6     218                      Sujo de óleo                      33   
                             2024-06-28   2145.60          169.3          124.1                  34.2     218                      Sujo d

### Resetando índex para criação dos gráficos e modelo

Retransformando a coluna de Total para integer

In [191]:
df_defeitos_medidas_geral_reset = df_defeitos_medidas_geral.reset_index()

df_defeitos_medidas_geral_reset['Total de Defeitos no Dia'] = df_defeitos_medidas_geral_reset['Total de Defeitos no Dia'].astype(np.int32)

In [192]:
df_defeitos_medidas_geral_reset

,Ordem de prod,Ref. do Artigo,Data de prod,Temperatura (F°),Peso médio (g),Comprimento médio (mm),Diâmetro,Total de Defeitos no Dia,Defeito Mais Frequente do Dia,Sujo de óleo,...,Bolha aberta interna,Cascão cortante,Espessura abaixo do mínimo,Fundo deformado,Bolha na alça,Ferrugem no corpo,Fagulha,Alça deformada,Instabilidade,Crú
0,192970,C0516.0000R,2024-01-03,2126.97,169.3,110.4,32.5,222,Sujo de óleo,31,...,0,0,0,0,0,0,0,0,0,0
1,192970,C0516.0000R,2024-01-04,2118.92,169.7,85.6,18.1,249,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
2,192970,C0516.0000R,2024-01-05,2125.00,173.6,140.5,32.9,268,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
3,192970,C0516.0000R,2024-01-06,2125.40,170.2,144.7,32.7,242,Sujo de óleo,73,...,0,0,0,0,0,0,0,0,0,0
4,192970,C0516.0000R,2024-01-07,2126.76,171.7,141.5,30.2,232,Sujo de óleo,49,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,194121,C0516.0000R,2024-06-26,2154.44,169.0,124.3,32.6,258,Sujo de óleo,40,...,1,0,0,0,0,0,0,0,0,0
196,194121,C0516.0000R,2024-06-27,2152.32,168.9,120.2,33.6,218,Sujo de óleo,75,...,0,0,0,0,0,0,0,0,0,0
197,194121,C0516.0000R,2024-06-28,2145.60,169.3,124.1,34.2,218,Sujo de óleo,54,...,0,0,0,0,0,0,0,0,0,0
198,194121,C0516.0000R,2024-06-29,2145.60,168.8,120.0,33.5,213,Sujo de óleo,54,...,0,0,0,0,0,0,1,0,0,0


## Gráficos

### Gráfico de Correlação

In [193]:
colunas_adicionais = ['Temperatura (F°)', 'Peso médio (g)', 'Comprimento médio (mm)', 'Diâmetro']
df_completo = df_defeitos_medidas_geral_reset[colunas_ordered.tolist() + colunas_adicionais]

correlacao_completa = df_completo.corr()

fig = px.imshow(
    correlacao_completa,
    labels=dict(x="Variáveis", y="Variáveis", color="Correlação"),
    x=correlacao_completa.columns,
    y=correlacao_completa.columns,
    title="Correlação entre Defeitos e Variáveis Adicionais"
)

fig.update_layout(
    template="plotly_dark",
    title_x=0.5,
    width=2500,
    height=2500 
)

fig.show()



> Temperatura afeta nos defeitos de 'Ombro', defeitos classificados como 'Maior'


### Gráfico de Dispersão: Analisando Sujo de Óleo

Sujo de óleo por Temperatura

In [194]:
px.scatter(data_frame=df_defeitos_medidas_geral_reset, 
        x='Sujo de óleo',
        y='Temperatura (F°)', 
        color='Temperatura (F°)', 
        color_continuous_scale='redor',
        labels={'Sujo de óleo': 'Defeitos sujo de óleo no dia'},
        size='Sujo de óleo')

Sujo de óleo por Diâmetro

In [195]:
px.scatter(data_frame=df_defeitos_medidas_geral_reset, 
        x='Sujo de óleo',
        y='Diâmetro', 
        color='Diâmetro', 
        color_continuous_scale='redor',
        labels={'Sujo de óleo': 'Defeitos sujo de óleo no dia'},
        size='Sujo de óleo')

Sujo de óleo por Comprimento médio

In [196]:
px.scatter(data_frame=df_defeitos_medidas_geral_reset, 
        x='Sujo de óleo',
        y='Comprimento médio (mm)', 
        color='Comprimento médio (mm)', 
        color_continuous_scale='redor',
        labels={'Sujo de óleo': 'Defeitos sujo de óleo no dia'},
        size='Sujo de óleo')

Sujo de óleo por Peso médio

In [197]:
px.scatter(data_frame=df_defeitos_medidas_geral_reset, 
        x='Sujo de óleo',
        y='Peso médio (g)', 
        color='Peso médio (g)', 
        color_continuous_scale='redor',
        labels={'Sujo de óleo': 'Defeitos sujo de óleo no dia'},
        size='Sujo de óleo')